In [8]:
# Federal AI Inventory Loading Notebook

import pandas as pd
import numpy as np
from neo4j import GraphDatabase
import logging
from typing import List, Dict, Tuple, Optional
import uuid
from datetime import datetime
import json

# Configuration
NEO4J_URI = "neo4j://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "kuxFc8HN"  # Update with your password
CSV_PATH = r"D:\Docker\neo4j\import\2024_Fed_AI_Inventory_condensed_v2.csv"

In [9]:
# Block 1: Verify Python Dependencies
def verify_dependencies():
    required_packages = {
        'neo4j': 'neo4j',
        'pandas': 'pandas',
        'numpy': 'numpy'
    }
    
    missing_packages = []
    for package, import_name in required_packages.items():
        try:
            __import__(import_name)
            print(f"✓ {package} is installed")
        except ImportError:
            missing_packages.append(package)
            print(f"✗ {package} is missing")
    
    if missing_packages:
        print("\nPlease install missing packages using:")
        print(f"pip install {' '.join(missing_packages)}")
        return False
    return True

In [10]:
# Block 2: Verify Neo4j Connection and AI Categories
def verify_neo4j_state(uri: str, user: str, password: str) -> bool:
   try:
       driver = GraphDatabase.driver(uri, auth=(user, password))

       with driver.session() as session:
           result = session.run("RETURN 'Connection test' as test")
           if result.single()['test'] != 'Connection test':
               print("✗ Neo4j connection failed")
               return False
           print("✓ Neo4j connection successful")

       with driver.session() as session:
           result = session.run("""
               MATCH (c:AICategory)
               RETURN count(c) as category_count
           """)
           category_count = result.single()['category_count']
           if category_count == 0:
              print("✗ No AI Categories found - please load categories first")
              return False
           print(f"✓ Found {category_count} AI Categories")

       with driver.session() as session:
           result = session.run("""
               MATCH (u:UseCase)
               RETURN count(u) as usecase_count
           """)
           usecase_count = result.single()['usecase_count']
           if usecase_count > 0:
               print(f"! Warning: Found {usecase_count} existing Use Cases")
           else:
               print("✓ No existing Use Cases found")

       with driver.session() as session:  
           result = session.run("MATCH (m:Metadata) RETURN count(m) as metadata_count")
           metadata_count = result.single()['metadata_count'] 
           print(f"✓ Found {metadata_count} Metadata nodes")

       with driver.session() as session:
           result = session.run("MATCH (v:Version) RETURN count(v) as version_count")
           version_count = result.single()['version_count']
           print(f"✓ Found {version_count} Version nodes")

       driver.close()
       return True

   except Exception as e:
       print(f"✗ Neo4j verification failed: {str(e)}")
       return False

In [11]:
# Block 3: Verify CSV Data
def verify_csv_data(csv_path: str) -> bool:
    try:
        # Check file exists
        if not os.path.exists(csv_path):
            print(f"✗ CSV file not found: {csv_path}")
            return False
        print(f"✓ CSV file found: {csv_path}")
        
        # Load and check structure
        df = pd.read_csv(csv_path)
        required_columns = [
            'Use Case Name', 'Agency', 'Abr', 'Bureau', 'Topic Area',
            'Dev Stage', 'Purpose Benefits', 'Outputs', 'System Name'
        ]
        
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            print(f"✗ Missing required columns: {missing_columns}")
            return False
        print("✓ All required columns present")
        
        # Basic data quality checks
        print("\nData Quality Summary:")
        print(f"- Total records: {len(df)}")
        print(f"- Unique agencies: {df['Agency'].nunique()}")
        print(f"- Unique use cases: {df['Use Case Name'].nunique()}")
        print(f"- Records missing agency: {df['Agency'].isna().sum()}")
        print(f"- Records missing use case name: {df['Use Case Name'].isna().sum()}")
        
        # Check for duplicate use case names
        duplicates = df['Use Case Name'].value_counts()
        duplicates = duplicates[duplicates > 1]
        if not duplicates.empty:
            print(f"\n! Warning: Found {len(duplicates)} duplicate use case names:")
            for name, count in duplicates.items():
                print(f"  - '{name}': {count} occurrences")
                
        return True
        
    except Exception as e:
        print(f"✗ CSV verification failed: {str(e)}")
        return False

In [12]:
# Block 4: Run All Verifications
def run_verifications(neo4j_uri: str, neo4j_user: str, neo4j_password: str, csv_path: str) -> bool:
    print("Starting verification checks...\n")
    
    print("1. Checking Python Dependencies:")
    if not verify_dependencies():
        return False
    print()
    
    print("2. Checking Neo4j State:")
    if not verify_neo4j_state(neo4j_uri, neo4j_user, neo4j_password):
        return False
    print()
    
    print("3. Checking CSV Data:")
    if not verify_csv_data(csv_path):
        return False
    print()
    
    print("✓ All verifications passed!")
    return True

In [13]:
# Run verifications before proceeding
import os
if run_verifications(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD, CSV_PATH):
    print("\nReady to proceed with data loading!")
else:
    print("\n✗ Please resolve verification issues before proceeding.")

Starting verification checks...

1. Checking Python Dependencies:
✓ neo4j is installed
✓ pandas is installed
✓ numpy is installed

2. Checking Neo4j State:
✓ Neo4j connection successful
✓ Found 14 AI Categories
✓ No existing Use Cases found
✓ Found 1 Metadata nodes
✓ Found 1 Version nodes

3. Checking CSV Data:
✓ CSV file found: D:\Docker\neo4j\import\2024_Fed_AI_Inventory_condensed_v2.csv
✓ All required columns present

Data Quality Summary:
- Total records: 2133
- Unique agencies: 42
- Unique use cases: 2044
- Records missing agency: 0
- Records missing use case name: 0

! Warning: Found 35 duplicate use case names:
  - 'Generative AI Usage': 51 occurrences
  - 'Veritone': 4 occurrences
  - 'Chatbot': 3 occurrences
  - 'ArcGIS': 3 occurrences
  - 'Evidence.com - Axon': 3 occurrences
  - 'Website Chatbot Assistant': 2 occurrences
  - 'Automating blood smear cell counts using machine learning': 2 occurrences
  - 'Machine learning for tsunami source zones': 2 occurrences
  - 'Predicting

In [14]:
# Enhanced logging setup
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class Neo4jConnection:
    """Manages Neo4j database connection and basic operations"""
    
    def __init__(self, uri: str, user: str, password: str):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        
    def close(self):
        """Close the driver connection"""
        self.driver.close()
        
    def verify_connection(self) -> bool:
        """Test database connection"""
        try:
            with self.driver.session() as session:
                result = session.run("RETURN 'Connection test' as test")
                return result.single()['test'] == 'Connection test'
        except Exception as e:
            logger.error(f"Connection failed: {str(e)}")
            return False
            
    def execute_query(self, query: str, parameters: dict = None) -> List[Dict]:
        """Execute a Cypher query and return results"""
        try:
            with self.driver.session() as session:
                result = session.run(query, parameters or {})
                return [record.data() for record in result]
        except Exception as e:
            logger.error(f"Query execution failed: {str(e)}")
            raise

class FedAIDataLoader:
    """Handles loading Federal AI Inventory data into Neo4j"""
    
    def __init__(self, connection: Neo4jConnection, csv_path: str):
        self.connection = connection
        self.csv_path = csv_path
        self.df = None
        
    def load_csv(self) -> None:
        """Load and prepare CSV data"""
        try:
            self.df = pd.read_csv(self.csv_path)
            logger.info(f"Loaded {len(self.df)} records from CSV")
        except Exception as e:
            logger.error(f"Failed to load CSV: {str(e)}")
            raise
            
    def create_constraints(self) -> None:
        """Create database constraints and indexes"""
        constraints = [
            "CREATE CONSTRAINT agency_name IF NOT EXISTS FOR (a:Agency) REQUIRE a.name IS UNIQUE",
            "CREATE CONSTRAINT bureau_name IF NOT EXISTS FOR (b:Bureau) REQUIRE b.name IS UNIQUE",
            "CREATE CONSTRAINT use_case_composite IF NOT EXISTS FOR (u:UseCase) REQUIRE (u.name, u.agency) IS UNIQUE",
            "CREATE CONSTRAINT system_name IF NOT EXISTS FOR (s:System) REQUIRE s.name IS UNIQUE",
            "CREATE INDEX use_case_topic IF NOT EXISTS FOR (u:UseCase) ON (u.topic_area)"
        ]
        
        for constraint in constraints:
            try:
                self.connection.execute_query(constraint)
            except Exception as e:
                logger.warning(f"Constraint creation warning: {str(e)}")

    def load_metadata(self) -> None:
        query = """
        MERGE (m:Metadata {version: '1.0'})
        MERGE (v:Version {number: '1.0'})
        MERGE (m)-[:CURRENT_VERSION]->(v)
        """
        self.connection.execute_query(query)
        logger.info("Loaded metadata and version")
                
    def load_agencies(self) -> None:
        """Load agencies and create relationships"""
        query = """
        UNWIND $agencies AS agency
        MERGE (a:Agency {name: agency.name})
        SET a.abbreviation = agency.abbreviation
        """
        
        agencies = self.df[['Agency', 'Abr']].drop_duplicates().to_dict('records')
        agencies = [{'name': a['Agency'], 'abbreviation': a['Abr']} for a in agencies]
        
        self.connection.execute_query(query, {'agencies': agencies})
        logger.info(f"Loaded {len(agencies)} agencies")
        
    def load_bureaus(self) -> None:
        """Load bureaus and link to agencies"""
        query = """
        UNWIND $bureaus AS bureau
        MATCH (a:Agency {name: bureau.agency})
        MERGE (b:Bureau {name: bureau.name})
        MERGE (a)-[:HAS_BUREAU]->(b)
        """
        
        bureaus = self.df[['Agency', 'Bureau']].dropna().drop_duplicates().to_dict('records')
        bureaus = [{'agency': b['Agency'], 'name': b['Bureau']} for b in bureaus]
        
        self.connection.execute_query(query, {'bureaus': bureaus})
        logger.info(f"Loaded {len(bureaus)} bureaus")
        
    def load_use_cases(self) -> None:
        """Load use cases with properties using composite key (name + agency)"""
        query = """
        UNWIND $use_cases AS uc
        MATCH (a:Agency {name: uc.agency})
        MERGE (u:UseCase {name: uc.name, agency: uc.agency})
        SET u.topic_area = uc.topic_area,
            u.dev_stage = uc.dev_stage,
            u.purpose_benefits = uc.purpose_benefits,
            u.outputs = uc.outputs,
            u.date_initiated = uc.date_initiated,
            u.contains_pii = uc.contains_pii,
            u.has_ato = uc.has_ato,
            u.infrastructure = uc.infrastructure,
            u.updated_at = datetime()
        MERGE (a)-[:HAS_USE_CASE]->(u)
        """
    
        use_cases = [{
            'agency': uc['Agency'],
            'name': uc['Use Case Name'], 
            'topic_area': uc['Topic Area'],
            'dev_stage': uc['Dev Stage'],
            'purpose_benefits': uc['Purpose Benefits'],
            'outputs': uc['Outputs'],
            'date_initiated': uc['Date Initiated'],
            'contains_pii': uc['Contains PII'] == 'Yes',
            'has_ato': uc['Has ATO'] == 'Yes',
            'infrastructure': uc['Infrastructure Provisioned']
        } for uc in self.df.to_dict('records')]
    
        batch_size = 100
        for i in range(0, len(use_cases), batch_size):
            batch = use_cases[i:i + batch_size]
            self.connection.execute_query(query, {'use_cases': batch})
            logger.info(f"Loaded use cases batch {i//batch_size + 1}")
            
    def load_systems(self) -> None:
        """Load systems and create relationships"""
        query = """
        UNWIND $systems AS sys
        MATCH (u:UseCase {name: sys.use_case, agency: sys.agency})
        MERGE (s:System {name: sys.name})
        MERGE (u)-[:USES_SYSTEM]->(s)
        """
        
        systems = self.df[['Use Case Name', 'Agency', 'System Name']].dropna().to_dict('records')
        systems = [{
            'use_case': s['Use Case Name'],
            'agency': s['Agency'],
            'name': s['System Name']
        } for s in systems]
        
        self.connection.execute_query(query, {'systems': systems})
        logger.info(f"Loaded {len(systems)} system relationships")
        
    def load_purpose_benefits(self) -> None:
        """Parse and load purpose/benefits"""
        query = """
        UNWIND $purposes AS p
        MATCH (u:UseCase {name: p.use_case, agency: p.agency})
        MERGE (pb:PurposeBenefit {description: p.description})
        MERGE (u)-[:HAS_PURPOSE]->(pb)
        """
        
        def parse_purposes(text):
            if pd.isna(text):
                return []
            return [p.strip() for p in text.split(';') if p.strip()]
        
        purposes = []
        for _, row in self.df.iterrows():
            for purpose in parse_purposes(row['Purpose Benefits']):
                purposes.append({
                    'use_case': row['Use Case Name'],
                    'agency': row['Agency'],
                    'description': purpose
                })
        
        self.connection.execute_query(query, {'purposes': purposes})
        logger.info(f"Loaded {len(purposes)} purpose/benefit relationships")
        
    def load_outputs(self) -> None:
        """Parse and load outputs"""
        query = """
        UNWIND $outputs AS out
        MATCH (u:UseCase {name: out.use_case, agency: out.agency})
        MERGE (o:Output {description: out.description})
        MERGE (u)-[:PRODUCES]->(o)
        """
        
        def parse_outputs(text):
            if pd.isna(text):
                return []
            return [o.strip() for o in text.split(';') if o.strip()]
        
        outputs = []
        for _, row in self.df.iterrows():
            for output in parse_outputs(row['Outputs']):
                outputs.append({
                    'use_case': row['Use Case Name'],
                    'agency': row['Agency'],
                    'description': output
                })
        
        self.connection.execute_query(query, {'outputs': outputs})
        logger.info(f"Loaded {len(outputs)} output relationships")

def verify_loading(connection: Neo4jConnection) -> None:
    """Verify the data loading results"""
    verification_queries = {
        "Use Cases": "MATCH (u:UseCase) RETURN count(u) as count",
        "Agencies": "MATCH (a:Agency) RETURN count(a) as count",
        "Systems": "MATCH (s:System) RETURN count(s) as count",
        "Purpose Benefits": "MATCH (p:PurposeBenefit) RETURN count(p) as count",
        "Outputs": "MATCH (o:Output) RETURN count(o) as count",
        "Use Cases with Systems": """
            MATCH (u:UseCase)-[:USES_SYSTEM]->()
            RETURN count(DISTINCT u) as count
        """,
        "Duplicate Check": """
            MATCH (u:UseCase)
            WITH u.name as name, count(*) as instances
            WHERE instances > 1
            RETURN name, instances
            ORDER BY instances DESC
            LIMIT 5
        """
    }
    
    print("\nVerification Results:")
    print("=" * 50)
    
    for description, query in verification_queries.items():
        try:
            result = connection.execute_query(query)
            if description != "Duplicate Check":
                print(f"{description}: {result[0]['count']}")
            else:
                if result:
                    print(f"\nDuplicate Use Cases (properly handled with different agencies):")
                    for record in result:
                        print(f"- {record['name']}: {record['instances']} instances")
                else:
                    print("\nNo duplicates found")
        except Exception as e:
            logger.error(f"Verification failed for {description}: {str(e)}")

def main():
    """Main execution function"""
    # Run initial verifications
    if not run_verifications(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD, CSV_PATH):
        logger.error("Verification failed. Please resolve issues before proceeding.")
        return
        
    # Initialize connection
    connection = Neo4jConnection(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
    
    try:
        # Initialize loader
        loader = FedAIDataLoader(connection, CSV_PATH)
        
        # Execute loading pipeline
        loader.load_csv()
        loader.create_constraints()
        
        # Load entities and relationships
        loader.load_metadata()
        loader.load_agencies()
        loader.load_bureaus()
        loader.load_use_cases()
        loader.load_systems()
        loader.load_purpose_benefits()
        loader.load_outputs()
        
        # Verify loading
        verify_loading(connection)
        
        logger.info("Data loading completed successfully")
        
    except Exception as e:
        logger.error(f"Loading failed: {str(e)}")
        raise
    finally:
        connection.close()

if __name__ == "__main__":
    main()

2025-02-05 09:05:50,844 - INFO - Loaded 2133 records from CSV


Starting verification checks...

1. Checking Python Dependencies:
✓ neo4j is installed
✓ pandas is installed
✓ numpy is installed

2. Checking Neo4j State:
✓ Neo4j connection successful
✓ Found 14 AI Categories
✓ No existing Use Cases found
✓ Found 1 Metadata nodes
✓ Found 1 Version nodes

3. Checking CSV Data:
✓ CSV file found: D:\Docker\neo4j\import\2024_Fed_AI_Inventory_condensed_v2.csv
✓ All required columns present

Data Quality Summary:
- Total records: 2133
- Unique agencies: 42
- Unique use cases: 2044
- Records missing agency: 0
- Records missing use case name: 0

! Warning: Found 35 duplicate use case names:
  - 'Generative AI Usage': 51 occurrences
  - 'Veritone': 4 occurrences
  - 'Chatbot': 3 occurrences
  - 'ArcGIS': 3 occurrences
  - 'Evidence.com - Axon': 3 occurrences
  - 'Website Chatbot Assistant': 2 occurrences
  - 'Automating blood smear cell counts using machine learning': 2 occurrences
  - 'Machine learning for tsunami source zones': 2 occurrences
  - 'Predicting

2025-02-05 09:05:51,062 - INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE CONSTRAINT agency_name IF NOT EXISTS FOR (e:Agency) REQUIRE (e.name) IS UNIQUE` has no effect.} {description: `CONSTRAINT agency_name FOR (e:Agency) REQUIRE (e.name) IS UNIQUE` already exists.} {position: None} for query: 'CREATE CONSTRAINT agency_name IF NOT EXISTS FOR (a:Agency) REQUIRE a.name IS UNIQUE'
2025-02-05 09:05:51,077 - INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE CONSTRAINT bureau_name IF NOT EXISTS FOR (e:Bureau) REQUIRE (e.name) IS UNIQUE` has no effect.} {description: `CONSTRAINT bureau_name FOR (e:Bureau) REQUIRE (e.name) IS UNIQUE` already exists.} {position: None} for query: 'CREATE CONSTRAINT bureau_name IF NOT EXISTS FOR (b:Bureau) REQUIRE 


Verification Results:
Use Cases: 2052
Agencies: 42
Systems: 322
Purpose Benefits: 2094
Outputs: 1709


2025-02-05 09:05:56,386 - INFO - Data loading completed successfully


Use Cases with Systems: 767

Duplicate Use Cases (properly handled with different agencies):
- Security Information and Event Management (SIEM) Alerting Models: 2 instances
- Grammarly: 2 instances
- Language Translation: 2 instances
- Chatbot: 2 instances
- Malware Reverse Engineering: 2 instances
